# MA3831 A3 WebCrawler and NLP System

In [229]:
# Import Libraries
import os
import requests
import pandas as pd
import nltk
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import urllib.parse
import json
from pygments import highlight, lexers, formatters
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
import pickle

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk import PorterStemmer

import time
import ast

Chose the https://www.imdb.com/chart/top/?ref_=nv_mv_250&sort=user_rating%2Cdesc website to look through, instead of the default "ranking" they use. Unsure as to what that ranking is based off so using user rating for each movie in descending order for the top 250. 

# Task 2: Web scrapping

In [9]:
# Functions for
def get_html(url):
    """Return html from given url."""
    page = requests.get(url)
    return page


def clean_html(page):
    """Clean html from website."""
    soup = BeautifulSoup(page.text, 'lxml')
    return soup


def print_html(soup):
    """Print html from website."""
    print(soup.prettify())
    
def extract_names_with_keyword(strings_list, keyword):
    return [' '.join(string.split()[1:]) for string in strings_list if keyword in string]

In [11]:
# Retrieve html from website
url = 'https://editorial.rottentomatoes.com/guide/best-movies-of-all-time/'
webpage = get_html(url)

In [12]:
webpage = clean_html(webpage)

In [13]:
# Print html from webpage
print_html(webpage)

<!DOCTYPE html>
<html class="hitim" lang="en-US">
 <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <!-- OneTrust Cookies Consent Notice start for rottentomatoes.com -->
  <script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() { }
  </script>
  <!-- OneTrust Cookies Consent Notice end for rottentomatoes.com -->
  <!-- OneTrust IAB US Privacy (USP) -->
  <script ccpa-opt-out-geo="US" ccpa-opt-out-ids="USP" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
  </script>
  <!-- OneTrust 

In [14]:
# retrieve movie titles from html
scraped_movies = webpage.find_all('a', class_='title')
print(scraped_movies)
len(scraped_movies)  # Expected 300 movies

[<a class="title" href="https://www.rottentomatoes.com/m/la_confidential">L.A. Confidential</a>, <a class="title" href="https://www.rottentomatoes.com/m/the_godfather">The Godfather</a>, <a class="title" href="https://www.rottentomatoes.com/m/1003707-casablanca">Casablanca</a>, <a class="title" href="https://www.rottentomatoes.com/m/seven_samurai_1956">Seven Samurai</a>, <a class="title" href="https://www.rottentomatoes.com/m/parasite_2019">Parasite</a>, <a class="title" href="https://www.rottentomatoes.com/m/schindlers_list">Schindler's List</a>, <a class="title" href="https://www.rottentomatoes.com/m/top_gun_maverick">Top Gun: Maverick</a>, <a class="title" href="https://www.rottentomatoes.com/m/toy_story_2">Toy Story 2</a>, <a class="title" href="https://www.rottentomatoes.com/m/chinatown">Chinatown</a>, <a class="title" href="https://www.rottentomatoes.com/m/on_the_waterfront">On the Waterfront</a>, <a class="title" href="https://www.rottentomatoes.com/m/the_battle_of_algiers">The 

300

In [15]:
# Get movie titles
titles = []

for title in scraped_movies:
    titles.append(title.get_text().strip())
print(titles)
len(titles)

['L.A. Confidential', 'The Godfather', 'Casablanca', 'Seven Samurai', 'Parasite', "Schindler's List", 'Top Gun: Maverick', 'Toy Story 2', 'Chinatown', 'On the Waterfront', 'The Battle of Algiers', 'Toy Story', 'Rear Window', 'Modern Times', 'How to Train Your Dragon', 'All About Eve', 'Spirited Away', 'Up', 'The Third Man', 'Spotlight', 'Spider-Man: Into the Spider-Verse', 'The Philadelphia Story', 'Finding Nemo', "Singin' in the Rain", '12 Angry Men', 'Toy Story 3', 'Sunset Boulevard', 'Coco', 'The Godfather, Part II', 'Three Colors: Red', 'Selma', 'Zootopia', 'Citizen Kane', 'Annie Hall', 'Cool Hand Luke', 'The Holdovers', 'Inside Out', 'Dr. Strangelove Or: How I Learned to Stop Worrying and Love the Bomb', 'Let the Right One In', 'The Lord of the Rings: The Two Towers', 'Knives Out', 'M', 'Toy Story 4', 'The Wrestler', 'Goodfellas', 'The Wizard of Oz', 'Double Indemnity', 'Psycho', 'Paddington 2', 'Before Sunrise', 'The Dark Knight', 'The Maltese Falcon', 'It Happened One Night', 'T

300

In [16]:
# Get URLS for movies
urls = []

for link in scraped_movies:
    urls.append(link.get('href'))
print(urls)
len(urls)

['https://www.rottentomatoes.com/m/la_confidential', 'https://www.rottentomatoes.com/m/the_godfather', 'https://www.rottentomatoes.com/m/1003707-casablanca', 'https://www.rottentomatoes.com/m/seven_samurai_1956', 'https://www.rottentomatoes.com/m/parasite_2019', 'https://www.rottentomatoes.com/m/schindlers_list', 'https://www.rottentomatoes.com/m/top_gun_maverick', 'https://www.rottentomatoes.com/m/toy_story_2', 'https://www.rottentomatoes.com/m/chinatown', 'https://www.rottentomatoes.com/m/on_the_waterfront', 'https://www.rottentomatoes.com/m/the_battle_of_algiers', 'https://www.rottentomatoes.com/m/toy_story', 'https://www.rottentomatoes.com/m/1017289-rear_window', 'https://www.rottentomatoes.com/m/modern_times', 'https://www.rottentomatoes.com/m/how_to_train_your_dragon', 'https://www.rottentomatoes.com/m/1000626-all_about_eve', 'https://www.rottentomatoes.com/m/spirited_away', 'https://www.rottentomatoes.com/m/up', 'https://www.rottentomatoes.com/m/the_third_man', 'https://www.rott

300

Have scrapped through the Rotten Tomatoes website and have gotten movie titles and the urls for each movie. Next need to scrap through each of the urls for each movie and get movie details like, released date, genre, rating, synopsis, director, 
Didn't take reviews as it doesnt relate to the movie genre


In [17]:
# Scrap through urls
movie_url = 'https://www.rottentomatoes.com/m/la_confidential'

movie = get_html(movie_url)
movie = clean_html(movie)
print(movie)

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/" xmlns="http://www.w3.org/1999/xhtml">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<script charset="UTF-8" crossorigin="anonymous" data-domain-script="7e979733-6841-4fce-9182-515fac69187f" integrity="sha384-TKdmlzVmoD70HzftTw4WtOzIBL5mNx8mXSRzEvwrWjpIJ7FZ/EuX758yMDWXtRUN" src="https://cdn.cookielaw.org/consent/7e979733-6841-4fce-9182-515fac69187f/otSDKStub.js" type="text/javascript">
</script>
<script type="text/javascript">
                function OptanonWrapper() { }
            </script>
<script ccpa-opt-out-geo="US" ccpa-opt-out-ids="USP" ccpa-opt-out-lspa="false" charset="UTF-8" src="https://cdn.cookielaw.org/opt-out/otCCPAiab.js" type="text/javascript">
</script>
<script src="/assets/pizza-pie/javascripts/bundles/roma/rt-common.js?single"></script>
<meta content="text/html; charset=utf-8" http-equiv="

In [95]:
info = movie.find_all('div', class_='category-wrap')
print(info)

# director = str(info.get('Director')).strip()
# print(director)

[<div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Director</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/krzysztof_kieslowski">Krzysztof Kieslowski</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Producer</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/marin_karmitz">Marin Karmitz</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Screenwriter</rt-text>
</dt>
<dd>
<rt-link href="/celebrity/krzysztof_kieslowski">Krzysztof Kieslowski</rt-link><rt-text class="delimiter">, </rt-text>
<rt-link href="/celebrity/krzysztof_piesiewicz">Krzysztof Piesiewicz</rt-link>
</dd>
</div>, <div class="category-wrap">
<dt class="key">
<rt-text class="key" size="0.875">Production Co</rt-text>
</dt>
<dd>
<rt-text>RTS - Radio Télévision Suisse</rt-text><rt-text class="delimiter">, </rt-text>
<rt-text>CAB Productions</rt-text><rt-text class="delimiter">, </rt-text>
<rt-text>

In [79]:
description = movie.find('div', class_='synopsis-wrap').text.strip()
name = ' '.join(description.split()[1:])

print(name)

Synopsis
Three policemen, each with his own motives and obsessions, tackle the corruption surrounding an unsolved murder at a downtown Los Angeles coffee shop in the early 1950s. Detective Lieutenant Exley (Guy Pearce), the son of a murdered detective, is out to avenge his father's killing. The ex-partner of Officer White (Russell Crowe), implicated in a scandal rooted out by Exley, was one of the victims. Sergeant Vincennes (Kevin Spacey) feeds classified information to a tabloid magnate (Danny DeVito).


In [128]:
movies = []

# for url in urls:
for url in urls:
    infos = []
    print(f"Getting {url}")
    director, rating, genre,synopsis = "", "", "", ""
    movie = get_html(url)
    movie = clean_html(movie)
    # get all data from the category-wrap class
    movie_details = movie.find_all('div', class_='category-wrap')
    # print(info)
    print("Appending to list")
    for row in movie_details:
        infos.append(row.get_text().strip().replace('\n', ' '))
    print("Getting movie details")
    try:
        director = extract_names_with_keyword(strings_list = infos, keyword = "Director")
        print(f"The director is: {director}")  
    except:
        pass
    try:
        rating = extract_names_with_keyword(strings_list = infos, keyword = "Rating")
        print(f"The rating is: {rating}")
    except:
        pass
    try:
        genre = extract_names_with_keyword(strings_list = infos, keyword = "Genre")
        print(f"The genre is: {genre}")
    except:
        pass
    try:
        section = movie.find('div', class_='synopsis-wrap').text.strip()
        synopsis = ' '.join(section.split()[1:])
        print(f"Synopsis: {synopsis}")
    except:
        pass
    if url !="":
        info_vector = [director,rating, genre,synopsis]
        #making a list of lists
        movies.append(info_vector)
        time.sleep(10)
        print(f"{len(movies)} movies found")
        
print(movies)

Getting https://www.rottentomatoes.com/m/la_confidential
Appending to list
Getting movie details
The director is: ['Curtis Hanson']
The rating is: ['R']
The genre is: ['Crime, Drama']
Synopsis: Three policemen, each with his own motives and obsessions, tackle the corruption surrounding an unsolved murder at a downtown Los Angeles coffee shop in the early 1950s. Detective Lieutenant Exley (Guy Pearce), the son of a murdered detective, is out to avenge his father's killing. The ex-partner of Officer White (Russell Crowe), implicated in a scandal rooted out by Exley, was one of the victims. Sergeant Vincennes (Kevin Spacey) feeds classified information to a tabloid magnate (Danny DeVito).
1 movies found
Getting https://www.rottentomatoes.com/m/the_godfather
Appending to list
Getting movie details
The director is: ['Francis Ford Coppola']
The rating is: ['R']
The genre is: ['Crime, Drama']
Synopsis: Widely regarded as one of the greatest films of all time, this mob drama, based on Mario Pu

In [129]:
print(len(movies))

300


In [151]:
df = pd.DataFrame(titles, columns=["titles"])
df

,titles
0,L.A. Confidential
1,The Godfather
2,Casablanca
3,Seven Samurai
4,Parasite
...,...
295,Beauty and the Beast
296,The Killing
297,The Rules of the Game
298,Eyes Without a Face


Receives all information. Need to remove html and just leave headers and content for each one. The synopsis is in a different class then the rest of the movie information. So 2 different find_all's were used and will need to be used per movie.

In [152]:
df['urls'] = urls
df

,titles,urls
0,L.A. Confidential,https://www.rottentomatoes.com/m/la_confidential
1,The Godfather,https://www.rottentomatoes.com/m/the_godfather
2,Casablanca,https://www.rottentomatoes.com/m/1003707-casab...
3,Seven Samurai,https://www.rottentomatoes.com/m/seven_samurai...
4,Parasite,https://www.rottentomatoes.com/m/parasite_2019
...,...,...
295,Beauty and the Beast,https://www.rottentomatoes.com/m/1001902-beaut...
296,The Killing,https://www.rottentomatoes.com/m/killing
297,The Rules of the Game,https://www.rottentomatoes.com/m/the_rules_of_...
298,Eyes Without a Face,https://www.rottentomatoes.com/m/eyes_without_...


In [153]:
df_new = pd.DataFrame(movies, columns=["director", "rating", "genre", "synopsis"])
df_new

,director,rating,genre,synopsis
0,[Curtis Hanson],[],"[Crime, Drama]","Three policemen, each with his own motives and..."
1,[Francis Ford Coppola],[],"[Crime, Drama]",Widely regarded as one of the greatest films o...
2,[Michael Curtiz],[],[Drama],"Rick Blaine (Humphrey Bogart), who owns a nigh..."
3,[Akira Kurosawa],[],[Action],A samurai answers a village's request for prot...
4,[Bong Joon Ho],[],"[Comedy, Mystery & Thriller, Drama]",Greed and class discrimination threaten the ne...
...,...,...,...,...
295,[Jean Cocteau],[],[Fantasy],The story of a gentle-hearted beast in love wi...
296,[Stanley Kubrick],[],"[Crime, Drama]",Career criminal Johnny Clay (Sterling Hayden) ...
297,[Jean Renoir],[],"[Comedy, Drama]","André is having an affair with Christine, whos..."
298,[Georges Franju],[],"[Horror, Drama]",Dr. Génessier (Pierre Brasseur) is riddled wit...


In [154]:
df = pd.concat([df, df_new], axis=1)
df

,titles,urls,director,rating,genre,synopsis
0,L.A. Confidential,https://www.rottentomatoes.com/m/la_confidential,[Curtis Hanson],[],"[Crime, Drama]","Three policemen, each with his own motives and..."
1,The Godfather,https://www.rottentomatoes.com/m/the_godfather,[Francis Ford Coppola],[],"[Crime, Drama]",Widely regarded as one of the greatest films o...
2,Casablanca,https://www.rottentomatoes.com/m/1003707-casab...,[Michael Curtiz],[],[Drama],"Rick Blaine (Humphrey Bogart), who owns a nigh..."
3,Seven Samurai,https://www.rottentomatoes.com/m/seven_samurai...,[Akira Kurosawa],[],[Action],A samurai answers a village's request for prot...
4,Parasite,https://www.rottentomatoes.com/m/parasite_2019,[Bong Joon Ho],[],"[Comedy, Mystery & Thriller, Drama]",Greed and class discrimination threaten the ne...
...,...,...,...,...,...,...
295,Beauty and the Beast,https://www.rottentomatoes.com/m/1001902-beaut...,[Jean Cocteau],[],[Fantasy],The story of a gentle-hearted beast in love wi...
296,The Killing,https://www.rottentomatoes.com/m/killing,[Stanley Kubrick],[],"[Crime, Drama]",Career criminal Johnny Clay (Sterling Hayden) ...
297,The Rules of the Game,https://www.rottentomatoes.com/m/the_rules_of_...,[Jean Renoir],[],"[Comedy, Drama]","André is having an affair with Christine, whos..."
298,Eyes Without a Face,https://www.rottentomatoes.com/m/eyes_without_...,[Georges Franju],[],"[Horror, Drama]",Dr. Génessier (Pierre Brasseur) is riddled wit...


In [136]:
export_csv = df.to_csv ('movies_df.csv', index = None, header=True)

In [157]:
df = pd.read_csv('movies_df.csv')

In [158]:
df.head(10)


,titles,urls,director,rating,genre,synopsis
0,L.A. Confidential,https://www.rottentomatoes.com/m/la_confidential,['Curtis Hanson'],['R'],"['Crime, Drama']","Three policemen, each with his own motives and..."
1,The Godfather,https://www.rottentomatoes.com/m/the_godfather,['Francis Ford Coppola'],['R'],"['Crime, Drama']",Widely regarded as one of the greatest films o...
2,Casablanca,https://www.rottentomatoes.com/m/1003707-casab...,['Michael Curtiz'],['PG'],['Drama'],"Rick Blaine (Humphrey Bogart), who owns a nigh..."
3,Seven Samurai,https://www.rottentomatoes.com/m/seven_samurai...,['Akira Kurosawa'],[],['Action'],A samurai answers a village's request for prot...
4,Parasite,https://www.rottentomatoes.com/m/parasite_2019,['Bong Joon Ho'],['R (Sexual Content|Language|Some Violence)'],"['Comedy, Mystery & Thriller, Drama']",Greed and class discrimination threaten the ne...
5,Schindler's List,https://www.rottentomatoes.com/m/schindlers_list,['Steven Spielberg'],['R (Language|Actuality Violence|Some Sexualit...,"['History, Drama']",Businessman Oskar Schindler (Liam Neeson) arri...
6,Top Gun: Maverick,https://www.rottentomatoes.com/m/top_gun_maverick,['Joseph Kosinski'],['PG-13 (Some Strong Language|Sequences of Int...,"['Action, Adventure']",After more than thirty years of service as one...
7,Toy Story 2,https://www.rottentomatoes.com/m/toy_story_2,"['Ash Brannon, John Lasseter, Lee Unkrich']",['G'],"['Kids & Family, Comedy, Adventure, Fantasy, A...",Woody (Tom Hanks) is stolen from his home by t...
8,Chinatown,https://www.rottentomatoes.com/m/chinatown,['Roman Polanski'],['R'],"['Crime, Drama']","When Los Angeles private eye J.J. ""Jake"" Gitte..."
9,On the Waterfront,https://www.rottentomatoes.com/m/on_the_waterf...,['Elia Kazan'],[],['Drama'],Dockworker Terry Malloy (Marlon Brando) had be...


In [ ]:
df.shape

In [161]:
df['rating'].value_counts()

rating
[]                                                                   92
['R']                                                                39
['PG']                                                               32
['G']                                                                19
['PG-13']                                                             6
                                                                     ..
['R (Some Sexual Material|Language)']                                 1
['PG-13 (Seq of Intense Action Violence|Frightening Images)']         1
['PG (Some Language|Animated Action Violence|Thematic Elements)']     1
['R (Intense Sequences of Violence|Disturbing Images)']               1
['R (Some Drug Use|Language Throughout|Some Violence)']               1
Name: count, Length: 114, dtype: int64

In [162]:
df["genre"].value_counts()

genre
['Drama']                                     48
['Comedy']                                    17
['Crime, Drama']                              15
['Comedy, Drama']                             12
['Mystery & Thriller']                        10
                                              ..
['War, Drama, Mystery & Thriller, Action']     1
['Drama, Biography']                           1
['Sci-Fi, Action, Mystery & Thriller']         1
['Sci-Fi, Adventure']                          1
['Horror, Fantasy, Mystery & Thriller']        1
Name: count, Length: 120, dtype: int64

In [163]:
df['director'].value_counts()

director
['Alfred Hitchcock']            7
['Akira Kurosawa']              6
['Billy Wilder']                6
['Steven Spielberg']            5
['Richard Linklater']           4
                               ..
['George Lucas']                1
['John Crowley']                1
['Jon Watts']                   1
['Rainer Werner Fassbinder']    1
['Robert Wiene']                1
Name: count, Length: 213, dtype: int64

In [187]:
# Remove extra characters from the ratings column
ratings = []
for row in df['rating']:
    new_rating = str(row).replace("[","").replace("]","").replace("'","").strip()
    ratings.append(new_rating)
ratings

['R',
 'R',
 'PG',
 '',
 'R (Sexual Content|Language|Some Violence)',
 'R (Language|Actuality Violence|Some Sexuality)',
 'PG-13 (Some Strong Language|Sequences of Intense Action)',
 'G',
 'R',
 '',
 '',
 'G',
 'PG',
 'G',
 'PG (Sequences of Intense Action|Brief Mild Language|Some Scary Images)',
 '',
 'PG (Some Scary Moments)',
 'PG (Some Peril and Action)',
 '',
 'R (Some Language|Sexual References)',
 'PG (Mild Language|Frenetic Action Violence|Thematic Elements)',
 '',
 'G',
 'G',
 '',
 'G',
 '',
 'PG (Thematic Elements)',
 'R',
 'R (Brief Strong Sex Scene)',
 'PG-13 (Brief Strong Language|A Suggestive Moment|Disturbing Thematic Material|Violence)',
 'PG (Rude Humor|Action|Some Thematic Elements)',
 'PG',
 'PG',
 'PG',
 'R (Language|Brief Sexual Material|Some Drug Use)',
 'PG (Some Action|Mild Thematic Elements)',
 'PG',
 'R (Brief Nudity|Bloody Violence|Disturbing Images|Language)',
 'PG-13 (Scary Images|Epic Battle Sequences)',
 'PG-13 (Drug Material|Brief Violence|Sexual Referen

In [188]:
df['rating'] = ratings
df

,titles,urls,director,rating,genre,synopsis
0,L.A. Confidential,https://www.rottentomatoes.com/m/la_confidential,['Curtis Hanson'],R,"['Crime, Drama']","Three policemen, each with his own motives and..."
1,The Godfather,https://www.rottentomatoes.com/m/the_godfather,['Francis Ford Coppola'],R,"['Crime, Drama']",Widely regarded as one of the greatest films o...
2,Casablanca,https://www.rottentomatoes.com/m/1003707-casab...,['Michael Curtiz'],PG,['Drama'],"Rick Blaine (Humphrey Bogart), who owns a nigh..."
3,Seven Samurai,https://www.rottentomatoes.com/m/seven_samurai...,['Akira Kurosawa'],,['Action'],A samurai answers a village's request for prot...
4,Parasite,https://www.rottentomatoes.com/m/parasite_2019,['Bong Joon Ho'],R (Sexual Content|Language|Some Violence),"['Comedy, Mystery & Thriller, Drama']",Greed and class discrimination threaten the ne...
...,...,...,...,...,...,...
295,Beauty and the Beast,https://www.rottentomatoes.com/m/1001902-beaut...,['Jean Cocteau'],,['Fantasy'],The story of a gentle-hearted beast in love wi...
296,The Killing,https://www.rottentomatoes.com/m/killing,['Stanley Kubrick'],,"['Crime, Drama']",Career criminal Johnny Clay (Sterling Hayden) ...
297,The Rules of the Game,https://www.rottentomatoes.com/m/the_rules_of_...,['Jean Renoir'],,"['Comedy, Drama']","André is having an affair with Christine, whos..."
298,Eyes Without a Face,https://www.rottentomatoes.com/m/eyes_without_...,['Georges Franju'],,"['Horror, Drama']",Dr. Génessier (Pierre Brasseur) is riddled wit...


In [191]:
df['rating'].value_counts()

rating
                                                                 92
R                                                                39
PG                                                               32
G                                                                19
PG-13                                                             6
                                                                 ..
R (Some Sexual Material|Language)                                 1
PG-13 (Seq of Intense Action Violence|Frightening Images)         1
PG (Some Language|Animated Action Violence|Thematic Elements)     1
R (Intense Sequences of Violence|Disturbing Images)               1
R (Some Drug Use|Language Throughout|Some Violence)               1
Name: count, Length: 114, dtype: int64

In [205]:
ratings = []
for row in df['rating']:
    val = str(row).split()
    # item = val.pop()
    # print(item)
    try:
        item = val.pop(0)
    
    except IndexError:
        item = ""
    ratings.append(item)
    
ratings  

['R',
 'R',
 'PG',
 '',
 'R',
 'R',
 'PG-13',
 'G',
 'R',
 '',
 '',
 'G',
 'PG',
 'G',
 'PG',
 '',
 'PG',
 'PG',
 '',
 'R',
 'PG',
 '',
 'G',
 'G',
 '',
 'G',
 '',
 'PG',
 'R',
 'R',
 'PG-13',
 'PG',
 'PG',
 'PG',
 'PG',
 'R',
 'PG',
 'PG',
 'R',
 'PG-13',
 'PG-13',
 '',
 'G',
 'R',
 'R',
 'G',
 '',
 'R',
 'PG',
 'R',
 'PG-13',
 '',
 '',
 '',
 '',
 '',
 'R',
 'R',
 'R',
 '',
 'PG-13',
 'R',
 '',
 '',
 '',
 'R',
 'PG-13',
 'R',
 '',
 'R',
 'PG-13',
 'R',
 '',
 'R',
 '',
 'G',
 '',
 'R',
 '',
 'R',
 'PG',
 'R',
 'PG-13',
 'R',
 'R',
 '',
 '',
 '',
 '',
 'G',
 '',
 '',
 '',
 '',
 'R',
 '',
 'TV-14',
 'R',
 'R',
 'PG-13',
 'R',
 'R',
 'PG-13',
 'PG',
 '',
 'R',
 '',
 '',
 'R',
 '',
 '',
 'R',
 'PG-13',
 'R',
 '',
 'PG',
 '',
 'R',
 'G',
 'PG-13',
 'PG',
 'PG',
 'PG',
 'PG',
 'G',
 'PG-13',
 '',
 'PG-13',
 '',
 'PG',
 '',
 'R',
 'PG',
 'R',
 'PG',
 'R',
 'R',
 'PG',
 'PG',
 '',
 'PG-13',
 'PG-13',
 'R',
 '',
 'PG-13',
 'PG',
 'PG',
 'PG',
 'PG-13',
 '',
 '',
 '',
 '',
 '',
 'PG-13',
 'R',
 

In [206]:
df['rating'] = ratings
df

,titles,urls,director,rating,genre,synopsis
0,L.A. Confidential,https://www.rottentomatoes.com/m/la_confidential,['Curtis Hanson'],R,"['Crime, Drama']","Three policemen, each with his own motives and..."
1,The Godfather,https://www.rottentomatoes.com/m/the_godfather,['Francis Ford Coppola'],R,"['Crime, Drama']",Widely regarded as one of the greatest films o...
2,Casablanca,https://www.rottentomatoes.com/m/1003707-casab...,['Michael Curtiz'],PG,['Drama'],"Rick Blaine (Humphrey Bogart), who owns a nigh..."
3,Seven Samurai,https://www.rottentomatoes.com/m/seven_samurai...,['Akira Kurosawa'],,['Action'],A samurai answers a village's request for prot...
4,Parasite,https://www.rottentomatoes.com/m/parasite_2019,['Bong Joon Ho'],R,"['Comedy, Mystery & Thriller, Drama']",Greed and class discrimination threaten the ne...
...,...,...,...,...,...,...
295,Beauty and the Beast,https://www.rottentomatoes.com/m/1001902-beaut...,['Jean Cocteau'],,['Fantasy'],The story of a gentle-hearted beast in love wi...
296,The Killing,https://www.rottentomatoes.com/m/killing,['Stanley Kubrick'],,"['Crime, Drama']",Career criminal Johnny Clay (Sterling Hayden) ...
297,The Rules of the Game,https://www.rottentomatoes.com/m/the_rules_of_...,['Jean Renoir'],,"['Comedy, Drama']","André is having an affair with Christine, whos..."
298,Eyes Without a Face,https://www.rottentomatoes.com/m/eyes_without_...,['Georges Franju'],,"['Horror, Drama']",Dr. Génessier (Pierre Brasseur) is riddled wit...


In [210]:
df['rating'].value_counts()

rating
         92
R        88
PG       56
PG-13    44
G        19
TV-14     1
Name: count, dtype: int64

In [244]:
genres = []
# list_of_lists = [[value] for value in df["genre"]]
for row in df["genre"]:
    # print(row)
    val = str(row).replace("[","").replace("]","").replace("'","").strip()
    split_values = val.split(', ')
    # print(split_values)
    list_of_lists = [[value] for value in split_values]
    # print(list_of_lists)
    genres.append(list_of_lists)
print(genres)

[[['Crime'], ['Drama']], [['Crime'], ['Drama']], [['Drama']], [['Action']], [['Comedy'], ['Mystery & Thriller'], ['Drama']], [['History'], ['Drama']], [['Action'], ['Adventure']], [['Kids & Family'], ['Comedy'], ['Adventure'], ['Fantasy'], ['Animation']], [['Crime'], ['Drama']], [['Drama']], [['War'], ['History'], ['Drama']], [['Kids & Family'], ['Comedy'], ['Adventure'], ['Fantasy'], ['Animation']], [['Mystery & Thriller']], [['Comedy']], [['Kids & Family'], ['Fantasy'], ['Adventure'], ['Comedy'], ['Animation']], [['Drama']], [['Fantasy'], ['Adventure'], ['Animation']], [['Kids & Family'], ['Comedy'], ['Adventure'], ['Animation']], [['Mystery & Thriller']], [['Drama']], [['Kids & Family'], ['Action'], ['Adventure'], ['Comedy'], ['Fantasy'], ['Animation']], [['Romance'], ['Comedy']], [['Kids & Family'], ['Comedy'], ['Adventure'], ['Animation']], [['Musical'], ['Comedy']], [['Drama']], [['Kids & Family'], ['Comedy'], ['Adventure'], ['Fantasy'], ['Animation']], [['Drama']], [['Kids & Fam

In [245]:
df['genre'] = genres
df

,titles,urls,director,rating,genre,synopsis
0,L.A. Confidential,https://www.rottentomatoes.com/m/la_confidential,['Curtis Hanson'],R,"[[Crime], [Drama]]","Three policemen, each with his own motives and..."
1,The Godfather,https://www.rottentomatoes.com/m/the_godfather,['Francis Ford Coppola'],R,"[[Crime], [Drama]]",Widely regarded as one of the greatest films o...
2,Casablanca,https://www.rottentomatoes.com/m/1003707-casab...,['Michael Curtiz'],PG,[[Drama]],"Rick Blaine (Humphrey Bogart), who owns a nigh..."
3,Seven Samurai,https://www.rottentomatoes.com/m/seven_samurai...,['Akira Kurosawa'],,[[Action]],A samurai answers a village's request for prot...
4,Parasite,https://www.rottentomatoes.com/m/parasite_2019,['Bong Joon Ho'],R,"[[Comedy], [Mystery & Thriller], [Drama]]",Greed and class discrimination threaten the ne...
...,...,...,...,...,...,...
295,Beauty and the Beast,https://www.rottentomatoes.com/m/1001902-beaut...,['Jean Cocteau'],,[[Fantasy]],The story of a gentle-hearted beast in love wi...
296,The Killing,https://www.rottentomatoes.com/m/killing,['Stanley Kubrick'],,"[[Crime], [Drama]]",Career criminal Johnny Clay (Sterling Hayden) ...
297,The Rules of the Game,https://www.rottentomatoes.com/m/the_rules_of_...,['Jean Renoir'],,"[[Comedy], [Drama]]","André is having an affair with Christine, whos..."
298,Eyes Without a Face,https://www.rottentomatoes.com/m/eyes_without_...,['Georges Franju'],,"[[Horror], [Drama]]",Dr. Génessier (Pierre Brasseur) is riddled wit...


In [247]:
df['genre'].value_counts()

genre
[[Drama]]                                           48
[[Comedy]]                                          17
[[Crime], [Drama]]                                  15
[[Comedy], [Drama]]                                 12
[[Mystery & Thriller]]                              10
                                                    ..
[[War], [Drama], [Mystery & Thriller], [Action]]     1
[[Drama], [Biography]]                               1
[[Sci-Fi], [Action], [Mystery & Thriller]]           1
[[Sci-Fi], [Adventure]]                              1
[[Horror], [Fantasy], [Mystery & Thriller]]          1
Name: count, Length: 120, dtype: int64

## EDA

# Task 3: Data Wrangling

# Task 4: Machine Learning

### Text Cleaning Functions:

Justification of why sample was chosen

How does my topic relate to the real world. What are the real world applications? Proof of concept? 
Word2Vec TF-IDF
Functions for code

Word embedding
Ethics of webscrapper. Why chosen?

Beatiful soup why?

Improved Netflix recommendations from the model. 

In [ ]:
# Text cleaning functions

def remove_by_regex(texts, regexp):
    """Removes text by regexp from a list."""
    output_texts = regexp.sub("", texts)
    return output_texts


def remove_urls(input_text):
    """Remove urls from a string."""
    return remove_by_regex(input_text, re.compile(r"http.?://[^\s]+[\s]?"))


def text_preprocessing(uncleaned_text):
    """Preprocessing text by removing urls and punctuation."""
    review = re.sub('[^a-zA-Z]', ' ', uncleaned_text)
    review = review.lower()
    review = remove_urls(review)
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    return review

### Visualisation Functions:

In [ ]:
# A Variety of Visualisation Functions:



